## 232 - Parsing Winds and Temperatures Aloft Forecasts Part 2

[Youtube](https://www.youtube.com/watch?v=d8LHunYRVss)

In [1]:
l = "FT  3000    6000    9000   12000   18000   24000  30000  34000  39000"

In [2]:
end_char_indexes = [4, 9, 17, 25, 33, 41, 49, 56, 63, 70]

In [3]:
levels = []
for start_idx, end_idx in zip(end_char_indexes[:-1], end_char_indexes[1:]):
	levels.append(int(l[start_idx:end_idx]))

levels

[3000, 6000, 9000, 12000, 18000, 24000, 30000, 34000, 39000]

In [4]:
l = "BHM 1316 1915+07 2409+02 2832-03 2353-13 2458-25 237042 750852 761555"

In [5]:
data_groups = []
for start_idx, end_idx in zip(end_char_indexes[:-1], end_char_indexes[1:]):
	data_groups.append(l[start_idx:end_idx].strip())
	station = l[:3]

In [6]:
print(station)

BHM


In [7]:
print(data_groups)

['1316', '1915+07', '2409+02', '2832-03', '2353-13', '2458-25', '237042', '750852', '761555']


In [8]:
for level, dg in zip(levels, data_groups):
	wdir = int(dg[:2]) * 10
	wspd = int(dg[2:4])
	temperature = None

	if len(dg) > 4:
		temperature = int(dg[4:])
		if level > 24000:
			temperature *= -1
	print(wdir, wspd, temperature)

130 16 None
190 15 7
240 9 2
280 32 -3
230 53 -13
240 58 -25
230 70 -42
750 8 -52
760 15 -55


In [9]:
def parse_wind_temperature_forecast(fname):
	"""
	Parses AWC wind temperature forecast tables.
	"""
	sites = {}
	end_char_indexes = [4, 9, 17, 25, 33, 41, 49, 56, 63, 70]

	with open(fname) as f:
		lines = f.readlines()

	# Read the levels from the first line
	levels = []
	l = lines.pop(0).strip()
	for start_idx, end_idx in zip(end_char_indexes[:-1], end_char_indexes[1:]):
		levels.append(int(l[start_idx:end_idx]))

	# Read each site - its own line
	for l in lines:
		data_groups = []
		wspds = []
		wdirs = []
		temps = []
		station = l[:3]
		
	# Extract data groups
	for start_idx, end_idx in zip(end_char_indexes[:-1], end_char_indexes[1:]):
		data_groups.append(l[start_idx:end_idx].strip())
		station = l[:3]
		
		# Parse data groups
		for level, dg in zip(levels, data_groups):
			dg = dg.strip()
			
		if dg == '':
			wdir = None
			wspd = None
			
		else:		
			wdir = int(dg[:2]) * 10
			wspd = int(dg[2:4])
			temperature = None
			
			if len(dg) > 4:
				temperature = int(dg[4:])
				if level > 24000:
					temperature *= -1
					
			# Check for light and variable winds
			if dg[:4] == '9900':
				wdir = None
				wspd = 0
				
		# Check for winds over 99kts
		if (wdir is not None) and (wspd is not None) and (wdir > 360):
			wdir -= 500
			wspd += 100
				
	wspds.append(wspd)
	wdirs.append(wdir)
	temps.append(temperature)
		
	sites[station] = {'levels': levels, 'wind_speed': wspds, 'wind_direction': wdirs, 'temperature': temps}


	return sites

In [10]:
parse_wind_temperature_forecast('winds.txt')

{'H61': {'levels': [3000,
   6000,
   9000,
   12000,
   18000,
   24000,
   30000,
   34000,
   39000],
  'wind_speed': [85],
  'wind_direction': [260],
  'temperature': [-56]}}

In [11]:
def parse_data_group(data_group, level):
	""" Parses a single data group at a given level. """
	
	# Clean any extra characteres
	data_group = data_group.strip()
		
	# Nothing to parse!
	if data_group == '':
		return None, None, None 

	# Break out text for winds and temperature
	wind_text = data_group[:4]
	if len(data_group) > 4:
		temperature_text = data_group[4:]
	else:
		temperature_text = None
		
	# Parse the temperature 
	if temperature_text:
		temperature = int(temperature_text)
		if level > 24000:
			temperature *= -1
			
	else:
		temperature = None
			
	# Parse Winds
	wdir = int(wind_text[:2]) * 10
	wspd = int(wind_text[2:4])
				
					
	# Check for light and variable winds
	if (wdir == 990) and (wspd == 0):
		return 0, None, temperature
				
	# Check for winds over 99kts
	if wdir > 360:
		wdir -= 500
		wspd += 100
				
	return wspd, wdir, temperature

In [12]:
def parse_station(line, levels):
	""" Parses all data groups in a station line for winds and temps aloft. """
	
	data_groups = []
	wspds = []
	wdirs = []
	temps = []
	station = line[:3]
	end_char_indexes = [4, 9, 17, 25, 33, 41, 49, 56, 63, 70]
	
	# Extract data groups
	for start_idx, end_idx in zip(end_char_indexes[:-1], end_char_indexes[1:]):
		data_groups.append(line[start_idx: end_idx].strip())
		station = line[:3]
	
	# Parse data groups
	for level, dg in zip(levels, data_groups):
		wspd, wdir, temperature = parse_data_group(dg, level)
		wspds.append(wspd)
		wdirs.append(wdir)
		temps.append(temperature)
	
	return station, wspds, wdirs, temps

In [13]:
def better_parse_forecast(fname):
	""" Parses a winds and temperature aloft file. """
	sites = {}
	end_char_indexes = [4, 9, 17, 25, 33, 41, 49, 56, 63, 70]
	
	with open(fname) as f:
		lines = f.readlines()
		
	# Read the levels from the first line
	levels = []
	l = lines.pop(0).strip()
	for start_idx, end_idx in zip(end_char_indexes[:-1], end_char_indexes[1:]):
		levels.append(int(l[start_idx:end_idx]))
	
	# Parse each station	
	for line in lines:
		# print(line)
		stn, wspds, wdirs, temps = parse_station(line, levels)
		sites[stn] = {'level': levels, 'wind_speed': wspds, 'wind_direction': wdirs, 'temperature': temps}

	return sites

In [14]:
better_parse_forecast('winds.txt')

{'BHM': {'level': [3000, 6000, 9000, 12000, 18000, 24000, 30000, 34000, 39000],
  'wind_speed': [5, 15, 13, 13, 37, 47, 58, 74, 96],
  'wind_direction': [360, 350, 340, 320, 290, 280, 270, 270, 260],
  'temperature': [None, 2, 1, -3, -18, -31, -46, -51, -54]},
 'HSV': {'level': [3000, 6000, 9000, 12000, 18000, 24000, 30000, 34000, 39000],
  'wind_speed': [5, 14, 14, 15, 37, 51, 60, 70, 104],
  'wind_direction': [320, 350, 330, 320, 290, 290, 280, 270, 260],
  'temperature': [None, 1, 1, -4, -19, -31, -47, -53, -56]},
 'MGM': {'level': [3000, 6000, 9000, 12000, 18000, 24000, 30000, 34000, 39000],
  'wind_speed': [9, 12, 16, 29, 40, 47, 69, 78, 104],
  'wind_direction': [10, 340, 330, 300, 280, 270, 270, 260, 260],
  'temperature': [None, 3, 2, -2, -17, -30, -43, -50, -51]},
 'MOB': {'level': [3000, 6000, 9000, 12000, 18000, 24000, 30000, 34000, 39000],
  'wind_speed': [12, 8, 19, 25, 42, 63, 73, 87, 113],
  'wind_direction': [40, 320, 300, 280, 270, 270, 270, 260, 260],
  'temperature':